In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib
import matplotlib.pylab as plt
plt.style.use('bmh')
from sklearn.linear_model import Ridge

In [2]:
df_test = pd.read_csv('./test.csv')
df_ex = pd.read_csv('./sample_submission.csv')
df_train = pd.read_csv('./sales_train.csv')
df_shop = pd.read_csv('./shops.csv')
df_items = pd.read_csv('./items.csv')
df_cat = pd.read_csv('./item_categories.csv')

In [3]:
# Get item sells by month
grouped = df_train.groupby(['item_id', 'shop_id', 'date_block_num'])
month_stat = grouped['item_cnt_day'].sum()

In [4]:
# Create train table with "item_cnt_month"
X = np.array([*month_stat.index.values])
Y = np.array(month_stat.values)
df_train_m = pd.DataFrame({'date_block_num':X[:,2], 'item_id':X[:,0], 'shop_id':X[:,1], 'item_cnt_m':Y})
df_train_m = pd.merge(df_train_m, df_items[['item_id', 'item_category_id']], on='item_id')
df_train_m = df_train_m.sort_values(by='date_block_num')
df_train_m.head()

,date_block_num,item_cnt_m,item_id,shop_id,item_category_id
747775,0,1.0,9607,46,40
830176,0,1.0,10904,27,55
321175,0,1.0,4244,2,23
321172,0,1.0,4244,0,23
1292935,0,1.0,17114,44,40


In [16]:
Y = df_train_m['item_cnt_m']
X0 = df_train_m[['item_id', 'shop_id', 'item_category_id', 'date_block_num']]

In [17]:
X0['date_block_num_2'] = X0['date_block_num']**2
X0['date_block_num_3'] = X0['date_block_num']**3
X0['date_block_num_4'] = X0['date_block_num']**4
X0['date_block_num_5'] = X0['date_block_num']**5
X0['date_block_num_6'] = X0['date_block_num']**6
X0.head()

,item_id,shop_id,item_category_id,date_block_num,date_block_num_2,date_block_num_3,date_block_num_4,date_block_num_5,date_block_num_6
747775,9607,46,40,0,0,0,0,0,0
830176,10904,27,55,0,0,0,0,0,0
321175,4244,2,23,0,0,0,0,0,0
321172,4244,0,23,0,0,0,0,0,0
1292935,17114,44,40,0,0,0,0,0,0


In [18]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score 
from sklearn.preprocessing import OneHotEncoder

In [19]:
encoder = OneHotEncoder(categorical_features=[0,1,2])

In [20]:
X = encoder.fit_transform(X0.values)

In [21]:
regressor = Ridge()

In [22]:
regressor.fit(X, Y)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

### Test data

In [23]:
df_ex.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [24]:
df_test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [50]:
df_test_a = pd.merge(df_test, df_items[['item_id', 'item_category_id']], on='item_id')
df_test_a = df_test_a.sort_values(by='ID')
df_test_a['date_block_num'] = [34]*len(df_test_a)
X0_test = df_test_a[['item_id', 'shop_id', 'item_category_id', 'date_block_num']]
X0_test['date_block_num_2'] = X0_test['date_block_num']**2
X0_test['date_block_num_3'] = X0_test['date_block_num']**3
X0_test['date_block_num_4'] = X0_test['date_block_num']**4
X0['date_block_num_5'] = X0_test['date_block_num']**5
X0['date_block_num_6'] = X0['date_block_num']**6

In [51]:
X_test = encoder.transform(X0_test.values)

In [53]:
Y_pred = regressor.predict(X_test)

In [63]:
df_ex['item_cnt_month'] = Y_pred

In [65]:
df_ex.to_csv('pred.csv')

In [3]:
df_pred = pd.read_csv('pred.csv')

In [7]:
df_pred = df_pred.drop(columns=['Unnamed: 0'])
df_pred = df_pred.set_index('ID')

In [9]:
df_pred.to_csv('pred.csv')